#Librerias

In [ ]:
pip install pysentimiento

     |████████████████████████████████| 312 kB 4.5 MB/s 
     |████████████████████████████████| 175 kB 40.4 MB/s 
     |████████████████████████████████| 4.0 MB 41.3 MB/s 
     |████████████████████████████████| 212 kB 51.7 MB/s 
     |████████████████████████████████| 136 kB 59.8 MB/s 
     |████████████████████████████████| 1.1 MB 45.2 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 127 kB 58.9 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.2 MB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 53.6 MB/s 
     |████████████████████████████████| 144 kB 56.7 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=da94ab8c3462d348a7944d0deb26cd0bd8bab4fe5193172c368a68057b8b2d48
  Stored in directory: /root/.cache/pip/whee

In [ ]:
pip install transformers

In [ ]:
import transformers
from transformers import AutoTokenizer
from pysentimiento.preprocessing import preprocess_tweet
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import seaborn as sns
import re
import string
from collections import Counter, defaultdict
import string
#import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

import torch
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: ignored

#Opinion-Polarity Analysis


 **Visualización de los datos**

In [ ]:
ls

Data_test/     __MACOSX/     train_data.csv
Data_test.zip  sample_data/  train_data.zip


In [ ]:
path_dataset = "./train_data.csv"
train_data = pd.read_csv(r"./train_data.csv", sep=',',encoding='latin1', engine ="python" )

In [ ]:
#Columnas
train_data.keys()

Index(['Title', 'Opinion', 'Polarity', 'Attraction'], dtype='object')

In [ ]:
opinion_attaction_data = train_data.drop(columns=['Title','Polarity'])

In [ ]:
train_data=train_data.drop(columns=['Title','Attraction'])

In [ ]:
#Nos quedamos solo con las columnas de Opinión y de Polaridad
train_data.head()

,Opinion,Polarity
0,"Piensen dos veces antes de ir a este hotel, te...",1
1,Cuatro de nosotros fuimos recientemente a Eddi...,1
2,seguiré corta y simple: limpieza\n- bad. Tengo...,1
3,Al reservar un hotel con multipropiedad Mayan ...,1
4,"No pierdan su tiempo ni dinero, venimos porque...",1


Existe un desbalance en las clases.

In [ ]:
#Informacion basica del texto
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30212 entries, 0 to 30211
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Opinion   30210 non-null  object
 1   Polarity  30212 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 472.2+ KB


In [ ]:
import torch
import tensorflow as tf

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
# Set the classes from 0 - 4 (needed for BERT)
train_data['Polarity'] =train_data['Polarity'].replace([1,2,3,4,5], [0,1,2,3,4])

In [ ]:
# Reset the index ((¿Para qué es esto?))
train_data.reset_index(inplace=True)
train_data.shape

(30212, 3)

In [ ]:
#Existen registros que no tienen opiniones
i=0
for op in train_data['Opinion']:
  #aux=re.findall('"([^"]*)"',op)
  if op != op:
    print(i)
  i=i+1

8054
29566


In [ ]:
#Borramos los datos que no tienen opinion
train_data=train_data.drop(index=[8054,29566])

Existen registros que tienen polaridad pero no opinión.

Ahora revisamos si alguna opinión tiene quotes.

In [ ]:
train_data.shape

(30210, 3)

In [ ]:
for op in train_data['Opinion']:
  aux=re.findall('"([^"]*)"',op)
  if aux:
    print(op)

In [ ]:
# Function for removing quotes in the reviews
def remove_quotes(dataframe):
    nw = []
    for op in dataframe['Opinion']:
        nw.append(op.strip('"'))
    dataframe['Opinion'] = nw

In [ ]:
# Remove the quotes in the reviews
remove_quotes(train_data)

In [ ]:
train_data.shape

(30210, 3)

#Modelo

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Split train data into train and test set (90% - 20%)
train_data_X, test_data_X, train_data_y, test_data_y = train_test_split(train_data['Opinion'], 
                                                    train_data['Polarity'],
                                                    test_size=0.2, 
                                                    random_state=1)

In [ ]:
#Generando el nuevo train_data y test_data
train_data = pd.concat([train_data_X, train_data_y], axis=1)
df_test = pd.concat([test_data_X, test_data_y], axis=1)

In [ ]:
print('Size of test dataset: ', df_test.shape)

Size of test dataset:  (6042, 2)


In [ ]:
# Split the  train dataset into train and dev sets (90% - 10%)
X_train = train_data['Opinion']
y_train = train_data['Polarity']            
X_train, X_dev, y_train, y_dev = train_test_split(train_data['Opinion'], 
                                                    train_data['Polarity'],
                                                    test_size=0.1, 
                                                    random_state=1)

In [ ]:
print('Size of train X: ', X_train.shape)

Size of train X:  (21751,)


In [ ]:
# Check the classes
label_list = np.unique(y_train.values)
print(label_list)

[0 1 2 3 4]


In [ ]:
# Check the size of the sets
print('Size of the train set:', X_train.size, y_train.size)
print('Size of the dev set:', X_dev.size, y_dev.size)

Size of the train set: 21751 21751
Size of the dev set: 2417 2417


In [ ]:
# Save the reviews in variables for later use
opiniones = X_train.values
puntajes = y_train.values

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('pysentimiento/robertuito-base-cased')

text = "Esto es un tweet estoy usando #Robertuito @pysentimiento 🤣"
preprocessed_text = preprocess_tweet(text)

tokenizer.tokenize(preprocessed_text)

Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

['▁Esto',
 '▁es',
 '▁un',
 '▁tweet',
 '▁estoy',
 '▁usando',
 '▁ro',
 'bert',
 'uito',
 '▁@usuario',
 '▁',
 '▁emoji',
 '▁cara',
 '▁revolviéndose',
 '▁de',
 '▁la',
 '▁risa',
 '▁emoji']

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig

# Create the tokenizer and the model (BETO in this case)
tokenizer = AutoTokenizer.from_pretrained('pysentimiento/robertuito-base-cased')

model = BertForSequenceClassification.from_pretrained(
    'pysentimiento/robertuito-base-cased', 
    num_labels = 5, # The number of output labels
    output_attentions = False, # Whether the model returns attentions weights
    output_hidden_states = True, # Whether the model returns all hidden-states
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Some weights of the model checkpoint at pysentimiento/robertuito-base-cased were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.11.attention.self.value.weight', 'roberta.encoder.layer.4.attention.self.key.bias', 'roberta.encoder.layer.4.attention.output.dense.weight', 'roberta.encoder.layer.6.attention.self.query.weight', 'roberta.encoder.layer.0.output.dense.bias', 'roberta.encoder.layer.3.intermediate.dense.bias', 'roberta.encoder.layer.5.output.LayerNorm.weight', 'roberta.encoder.layer.9.attention.output.dense.bias', 'roberta.embeddings.LayerNorm.weight', 'roberta.encoder.layer.7.output.LayerNorm.bias', 'roberta.encoder.layer.4.attention.self.value.bias', 'roberta.encoder.layer.4.intermediate.dense.bias', 'roberta.encoder.layer.11.attention.self.key.weight', 'roberta.encoder.layer.11.output.dense.weight', 'roberta.encoder.layer.10.intermediate.dense.weight', 'roberta.encoder.layer.7.attention.output.dense.weight', 'roberta.encoder.layer.8.attention

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs
input_ids = []
attention_masks = []

for opinion in opiniones:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        opinion,                   #Sentence to encode.
                        add_special_tokens = True, #Add '[CLS]' and '[SEP]'
                        max_length = 130,           #Pad & truncate all sentences.
                        truncation = True,
                        padding='max_length',
                        return_attention_mask = True,  #Construct attn. masks.
                        return_tensors = 'pt',     #Return pytorch tensors.
                        verbose = True #Print more information and warnings.
                   )
    
    #Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    #And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

#Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(puntajes)

#Print sentence 0, now as a list of IDs.
print('Original: ', opiniones[0])
print('Token IDs:', input_ids[0])

Original:  Tienes que ir... No es la gran cosa, está normal pero ve, está agradable aunque dura 15 minutos. Pero como dice el título es un must.
Token IDs: tensor([    0,  7522,   443,   986,   544,   628,   442,   446,   890, 11394,
          781,  2640,   635, 27653,   781, 19219,  2196,  5232,  1797, 15267,
         1117,   619,  1331,   466,  7099,   442,   472,  1336,  1933,     2,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [ ]:
print(labels.size())
print(attention_masks.shape)
print(input_ids.size())
print(labels.shape)

torch.Size([21751])
torch.Size([21751, 130])
torch.Size([21751, 130])
torch.Size([21751])


In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split

# Calculate the number of samples to include in each set
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

19,575 training samples
2,176 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

#The DataLoader needs to know our batch size for training, so we specify it 
#here. For fine-tuning BERT on a specific task, the authors recommend a batch 
#size of 8, 16, 32, 64, or 128
batch_size = 8

#Create the DataLoaders for our training and validation sets.
#We'll take training samples in random order
train_dataloader = DataLoader(
            train_dataset,  #The training samples.
            sampler = RandomSampler(train_dataset), #Select batches randomly
            batch_size = batch_size #Trains with this batch size
        )

#For validation the order doesn't matter, so we'll just read them sequentially
validation_dataloader = DataLoader(
            val_dataset, # The validation samples
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially
            batch_size = batch_size # Evaluate with this batch size
        )

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )

from transformers import get_linear_schedule_with_warmup

epochs = 1 #The BERT authors recommend 4 epochs

# Total number of training steps is [number of batches] x [number of epochs]
# (Note that this is not the same as the number of training samples)
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value
                                            num_training_steps = total_steps)

#Calculates the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs"
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Clip the norm of the gradients to 1.0
        # This is to help prevent the "exploding gradients" problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc
        optimizer.step()

        # Update the learning rate
        scheduler.step()

    # Calculate the average loss over all of the batches
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took
    training_time = format_time(time.time() - t0)

    print("")
    print("  Training epcoh took: {:}".format(training_time))
        
    #               Validation


    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training)
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took
    validation_time = format_time(time.time() - t0)
    
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch
    training_stats.append(
        {
            'epoch': epoch_i + 1,

            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...
  Batch    40  of  2,447.    Elapsed: 0:00:16.
  Batch    80  of  2,447.    Elapsed: 0:00:33.
  Batch   120  of  2,447.    Elapsed: 0:00:49.
  Batch   160  of  2,447.    Elapsed: 0:01:06.
  Batch   200  of  2,447.    Elapsed: 0:01:22.
  Batch   240  of  2,447.    Elapsed: 0:01:39.
  Batch   280  of  2,447.    Elapsed: 0:01:56.
  Batch   320  of  2,447.    Elapsed: 0:02:13.
  Batch   360  of  2,447.    Elapsed: 0:02:30.


In [ ]:
# Remove the quotes in the reviews 
#remove_quotes(df_test)

In [ ]:
# Report the number of sentences
print('Number of test sentences: {:,}\n'.format(df_test.shape[0]))

# Create sentence and label lists
opiniones = df_test['Opinion'].values
#puntajes = y_dev.values

# Tokenize all of the sentences and map the tokens to thier word IDs
input_ids = []
attention_masks = []

for opinion in opiniones:
    encoded_dict = tokenizer.encode_plus(
                        opinion,                      # Sentence to encode
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 130,           # Pad & truncate all sentences
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attention masks
                        return_tensors = 'pt',     # Return pytorch tensors
                        truncation=True
                   )
    
    # Add the encoded sentence to the list  
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding)
    attention_masks.append(encoded_dict['attention_mask'])

#Convert the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

#Set the batch size
batch_size = 32  

#Create the DataLoader
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions = []

# Predict 
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    
    with torch.no_grad():
        result = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, return_dict=True)
        logits = result.logits
        logits = logits.detach().cpu().numpy()
    
        predictions.append(logits)

print('    DONE.')

In [ ]:
from sklearn.metrics import mean_absolute_error
# Combine the results across all batches
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [ ]:
# Set the classes in a range between 0 - 4
df1 = pd.DataFrame(flat_predictions, columns=['Label'])
df1['Label'] = df1['Label'].replace([0,1,2,3,4], [1,2,3,4,5])

In [ ]:
classes = []
for i in df1['Label']:
    classes.append(str(i))

indexes = []
for i in range(len(flat_predictions)):
    indexes.append(str(i))

In [ ]:
# Save the predicted labels in a .txt file
with open('test.txt', 'w') as f:
    for clas, i in zip(classes, indexes):
        print('"sentiment"\t'+ '"' + i +  '"'+ "\t" +  '"'+ clas +  '"'+ "\n", file=f)

In [ ]:
indices = []
for i in range(len(df1)):
  indices.append(i)
print(len(indices))

4834


In [ ]:
df_test['Polarity'] = df_test['Polarity'].replace([0,1,2,3,4], [1,2,3,4,5])

In [ ]:
#Test: df_test
#Pred: df1
w_opinion = []
real = []
w_pred = []
for i in range(len(df1)):
  if list(df_test['Polarity'])[i] != df1['Label'][i]:
    w_opinion.append( list(df_test['Opinion'])[i])
    real.append( list(df_test['Polarity'])[i] )
    w_pred.append(df1['Label'][i])




In [ ]:
#Tabla de predicciones erroneas
wrong_pred = pd.DataFrame()
wrong_pred['Opinion'] = w_opinion
wrong_pred['Real'] = real
wrong_pred['Prediccion'] = w_pred

wrong_pred.head()

,Opinion,Real,Prediccion
0,Mi marido y yo comimos en Riveria Grill varias...,4,5
1,Teníamos los camarones con coco - delicioso! U...,4,5
2,Acabamos de volver del Samba Vallarta después ...,4,5
3,El personal es exelente las recamaristas y las...,4,5
4,Mi recomendación es ir en la mañana o no pasar...,5,3


#Opinion - Attraction Analysis

**Data visualization**

In [ ]:
opinion_attaction_data.head()

,Opinion,Attraction
0,"Piensen dos veces antes de ir a este hotel, te...",Hotel
1,Cuatro de nosotros fuimos recientemente a Eddi...,Restaurant
2,seguiré corta y simple: limpieza\n- bad. Tengo...,Hotel
3,Al reservar un hotel con multipropiedad Mayan ...,Hotel
4,"No pierdan su tiempo ni dinero, venimos porque...",Hotel


In [ ]:
opinion_attaction_data2 = opinion_attaction_data.copy()
opinion_attaction_data2 = opinion_attaction_data2['Attraction'].replace('Hotel',0).to_frame()
opinion_attaction_data2 = opinion_attaction_data2['Attraction'].replace('Restaurant',1).to_frame()
opinion_attaction_data2 = opinion_attaction_data2['Attraction'].replace('Attractive',2).to_frame()

In [ ]:
opinion_attaction_data = pd.concat([opinion_attaction_data, opinion_attaction_data2], axis=1)
opinion_attaction_data.columns = ['Opinion', 'Attraction', 'Label']

In [ ]:
opinion_attaction_data.head()

,Opinion,Attraction,Label
0,"Piensen dos veces antes de ir a este hotel, te...",Hotel,0
1,Cuatro de nosotros fuimos recientemente a Eddi...,Restaurant,1
2,seguiré corta y simple: limpieza\n- bad. Tengo...,Hotel,0
3,Al reservar un hotel con multipropiedad Mayan ...,Hotel,0
4,"No pierdan su tiempo ni dinero, venimos porque...",Hotel,0


In [ ]:
#Existen registros que no tienen opiniones
i=0
for op in opinion_attaction_data['Opinion']:
  #aux=re.findall('"([^"]*)"',op)
  if op != op:
    print(i)
  i=i+1

8054
29566


In [ ]:
#Borramos los datos que no tienen opinion
opinion_attaction_data=opinion_attaction_data.drop(index=[8054,29566])

Datos de entrenamiento:

In [ ]:
train_data2 = opinion_attaction_data.drop(columns=['Attraction'])
train_data2.head()

,Opinion,Label
0,"Piensen dos veces antes de ir a este hotel, te...",0
1,Cuatro de nosotros fuimos recientemente a Eddi...,1
2,seguiré corta y simple: limpieza\n- bad. Tengo...,0
3,Al reservar un hotel con multipropiedad Mayan ...,0
4,"No pierdan su tiempo ni dinero, venimos porque...",0


#Modelo

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Split train data into train and test set (90% - 20%)
train_data_X, test_data_X, train_data_y, test_data_y = train_test_split(train_data2['Opinion'], 
                                                    train_data2['Label'],
                                                    test_size=0.2, 
                                                    random_state=1)

In [ ]:
#Generando el nuevo train_data y test_data
train_data = pd.concat([train_data_X, train_data_y], axis=1)
df_test = pd.concat([test_data_X, test_data_y], axis=1)

In [ ]:
print("Shape of train data: ", train_data.shape)
print("Shape of test data: ", df_test.shape)


Shape of train data:  (24168, 2)
Shape of test data:  (6042, 2)


In [ ]:
train_data.head()

,Opinion,Label
17256,"Acabo de volver de Nayarit, el complejo está\n...",0
10483,El pescado zarandeado es espectacular!!! La at...,1
6332,"Primera ocasión en conocer el hotel, muy buena...",0
7920,Esta situada en un punto magnifico para pasear...,2
15403,Mi primera vez en este lugar y fue fantástico....,0


In [ ]:
# Split the  train dataset into train and dev sets (90% - 10%)
#X_train = train_data['Opinion']
#y_train = train_data['Label']            
X_train, X_dev, y_train, y_dev = train_test_split(train_data['Opinion'], 
                                                    train_data['Label'],
                                                    test_size=0.1, 
                                                    random_state=1)

In [ ]:
X_train.shape

(21751,)

In [ ]:
# Check the classes
label_list = np.unique(y_train.values)
print("Classes: ", label_list)

Classes:  [0 1 2]


In [ ]:
# Check the size of the sets
print('Size of the train set:', X_train.size, y_train.size)
print('Size of the dev set:', X_dev.size, y_dev.size)

Size of the train set: 21751 21751
Size of the dev set: 2417 2417


In [ ]:
# Save the reviews in variables for later use
opiniones = X_train.values
puntajes = y_train.values

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig

# Create the tokenizer and the model (BETO in this case)
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", do_lower_case=False)

model = BertForSequenceClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-cased", 
    num_labels = 5, # The number of output labels
    output_attentions = False, # Whether the model returns attentions weights
    output_hidden_states = True, # Whether the model returns all hidden-states
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs
input_ids = []
attention_masks = []

for opinion in opiniones:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        opinion,                   #Sentence to encode.
                        add_special_tokens = True, #Add '[CLS]' and '[SEP]'
                        max_length = 512,           #Pad & truncate all sentences.
                        truncation = True,
                        padding='max_length',
                        return_attention_mask = True,  #Construct attn. masks.
                        return_tensors = 'pt',     #Return pytorch tensors.
                        verbose = True #Print more information and warnings.
                   )
    
    #Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    #And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

#Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(puntajes)

#Print sentence 0, now as a list of IDs.
print('Original: ', opiniones[0])
print('Token IDs:', input_ids[0])

Original:  Tienes que ir... No es la gran cosa, está normal pero ve, está agradable aunque dura 15 minutos. Pero como dice el título es un must.
Token IDs: tensor([    4,  2666,  1038,  1628,  1009,  1009,  1009,  1125,  1058,  1030,
         1523,  2467,  1017,  1266,  5131,  1355,  1312,  1017,  1266,  7789,
         2803,  6120,  1860,  3214,  1009,  1448,  1184,  2429,  1040,  4236,
         1058,  1049,  4150, 30940,  1009,     5,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [ ]:
labels.size()

torch.Size([21751])

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split

# Calculate the number of samples to include in each set
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

19,575 training samples
2,176 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

#The DataLoader needs to know our batch size for training, so we specify it 
#here. For fine-tuning BERT on a specific task, the authors recommend a batch 
#size of 8, 16, 32, 64, or 128
batch_size = 8

#Create the DataLoaders for our training and validation sets.
#We'll take training samples in random order
train_dataloader = DataLoader(
            train_dataset,  #The training samples.
            sampler = RandomSampler(train_dataset), #Select batches randomly
            batch_size = batch_size #Trains with this batch size
        )

#For validation the order doesn't matter, so we'll just read them sequentially
validation_dataloader = DataLoader(
            val_dataset, # The validation samples
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially
            batch_size = batch_size # Evaluate with this batch size
        )

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )

from transformers import get_linear_schedule_with_warmup

epochs = 1 #The BERT authors recommend 4 epochs

# Total number of training steps is [number of batches] x [number of epochs]
# (Note that this is not the same as the number of training samples)
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value
                                            num_training_steps = total_steps)

#Calculates the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import os
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs"
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Clip the norm of the gradients to 1.0
        # This is to help prevent the "exploding gradients" problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc
        optimizer.step()

        # Update the learning rate
        scheduler.step()

    # Calculate the average loss over all of the batches
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took
    training_time = format_time(time.time() - t0)

    print("")
    print("  Training epcoh took: {:}".format(training_time))
        
    #               Validation


    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training)
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took
    validation_time = format_time(time.time() - t0)
    
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch
    training_stats.append(
        {
            'epoch': epoch_i + 1,

            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...
  Batch    40  of  2,447.    Elapsed: 0:01:16.
  Batch    80  of  2,447.    Elapsed: 0:02:32.
  Batch   120  of  2,447.    Elapsed: 0:03:48.
  Batch   160  of  2,447.    Elapsed: 0:05:04.
  Batch   200  of  2,447.    Elapsed: 0:06:20.
  Batch   240  of  2,447.    Elapsed: 0:07:35.
  Batch   280  of  2,447.    Elapsed: 0:08:51.
  Batch   320  of  2,447.    Elapsed: 0:10:07.
  Batch   360  of  2,447.    Elapsed: 0:11:23.
  Batch   400  of  2,447.    Elapsed: 0:12:39.
  Batch   440  of  2,447.    Elapsed: 0:13:55.
  Batch   480  of  2,447.    Elapsed: 0:15:11.
  Batch   520  of  2,447.    Elapsed: 0:16:26.
  Batch   560  of  2,447.    Elapsed: 0:17:42.
  Batch   600  of  2,447.    Elapsed: 0:18:58.
  Batch   640  of  2,447.    Elapsed: 0:20:14.
  Batch   680  of  2,447.    Elapsed: 0:21:30.
  Batch   720  of  2,447.    Elapsed: 0:22:46.
  Batch   760  of  2,447.    Elapsed: 0:24:02.
  Batch   800  of  2,447.    Elapsed: 0:25:18.
  Batch   840  of

In [ ]:
# Load the prediction dataset
#df_test = pd.read_excel('/data_test.xlsx')

In [ ]:
# Remove the quotes in the reviews 
#remove_quotes(df_test)

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# Report the number of sentences
print('Number of test sentences: {:,}\n'.format(df_test.shape[0]))

# Create sentence and label lists
opiniones = df_test['Opinion'].values
#puntajes = y_dev.values

# Tokenize all of the sentences and map the tokens to thier word IDs
input_ids = []
attention_masks = []

for opinion in opiniones:
    encoded_dict = tokenizer.encode_plus(
                        opinion,                      # Sentence to encode
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attention masks
                        return_tensors = 'pt',     # Return pytorch tensors
                        truncation=True
                   )
    
    # Add the encoded sentence to the list  
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding)
    attention_masks.append(encoded_dict['attention_mask'])

#Convert the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

#Set the batch size
batch_size = 32  

#Create the DataLoader
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions = []

# Predict 
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    
    with torch.no_grad():
        result = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, return_dict=True)
        logits = result.logits
        logits = logits.detach().cpu().numpy()
    
        predictions.append(logits)

print('    DONE.')

Number of test sentences: 6,042

Predicting labels for 6,042 test sentences...
    DONE.


In [ ]:
from sklearn.metrics import mean_absolute_error
# Combine the results across all batches
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [ ]:
# Set the classes in a range between 0 - 4
df1 = pd.DataFrame(flat_predictions, columns=['Pred'])
#df1['Pred'] = df1['Pred'].replace([0,1,2,3,4], [1,2,3,4,5])

In [ ]:
classes = []
for i in df1['Pred']:
    classes.append(str(i))

indexes = []
for i in range(len(flat_predictions)):
    indexes.append(str(i))

In [ ]:
# Save the predicted labels in a .txt file
with open('test2.txt', 'w') as f:
    for clas, i in zip(classes, indexes):
        print('"sentiment"\t'+ '"' + i +  '"'+ "\t" +  '"'+ clas +  '"'+ "\n", file=f)

In [ ]:
indices = []
for i in range(len(df1)):
  indices.append(i)
print(len(indices))

6042


In [ ]:
#Test: df_test
#Pred: df1
w_opinion = []
real = []
w_pred = []
for i in range(len(df1)):
  if list(df_test['Label'])[i] != df1['Pred'][i]:
    w_opinion.append( list(df_test['Opinion'])[i])
    real.append( list(df_test['Label'])[i] )
    w_pred.append(df1['Pred'][i])




In [ ]:
#Tabla de predicciones erroneas
wrong_pred = pd.DataFrame()
wrong_pred['Opinion'] = w_opinion
wrong_pred['Real'] = real
wrong_pred['Prediccion'] = w_pred

wrong_pred.head()

,Opinion,Real,Prediccion
0,Las instalaciones ya empiezan a mostrar los es...,0,1
1,"Muy buenos los desayunos comidas y cenas, igua...",0,1
2,Excelente servicio y buenas bebidas y comida. ...,0,1
3,Era tan bello y elegante. Se lo recomiendo a t...,0,1
4,"Superó todas mis expectativas. Buena comida, a...",0,1


In [ ]:
print("Total de predicciones erroneas: ", len(wrong_pred))

Total de predicciones erroneas:  98
